<a href="https://colab.research.google.com/github/EjenY-Poltavchiny/Transport-problem/blob/main/Transport_problem_code(Poltavtsev).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transportation Problem (method of potentials)

## Balancing

In [ ]:
import numpy as np
import time

In [ ]:
def get_balanced_tp(supply, demand, costs, penalties = None):
    total_supply = sum(supply)
    total_demand = sum(demand)
    
    if total_supply < total_demand:
        if penalties is None:
            raise Exception('Supply less than demand, penalties required')
        new_supply = supply + [total_demand - total_supply]
        new_costs = costs + [penalties]
        return new_supply, demand, new_costs
    if total_supply > total_demand:
        new_demand = demand + [total_supply - total_demand]
        new_costs = costs + [[0 for _ in demand]]
        return supply, new_demand, new_costs
    return supply, demand, costs

## North West Corner Method

In [ ]:
def north_west_corner(supply, demand):
    supply_copy = supply.copy()
    demand_copy = demand.copy()
    i = 0
    j = 0
    bfs = []
    while len(bfs) < len(supply) + len(demand) - 1:
        s = supply_copy[i]
        d = demand_copy[j]
        v = min(s, d)
        supply_copy[i] -= v
        demand_copy[j] -= v
        bfs.append(((i, j), v))
        if supply_copy[i] == 0 and i < len(supply) - 1:
            i += 1
        elif demand_copy[j] == 0 and j < len(demand) - 1:
            j += 1
    return bfs

In [ ]:
supply = [30, 70, 50]
demand = [40, 30, 40, 40]
bfs = north_west_corner(supply, demand)
print(bfs)

[((0, 0), 30), ((1, 0), 10), ((1, 1), 30), ((1, 2), 30), ((2, 2), 10), ((2, 3), 40)]


## Transportation Method of potentials

In [ ]:
def get_us_and_vs(bfs, costs):
    us = [None] * len(costs)
    vs = [None] * len(costs[0])
    us[0] = 0
    bfs_copy = bfs.copy()
    while len(bfs_copy) > 0:
        for index, bv in enumerate(bfs_copy):
            i, j = bv[0]
            if us[i] is None and vs[j] is None: continue
                
            cost = costs[i][j]
            if us[i] is None:
                us[i] = cost - vs[j]
            else: 
                vs[j] = cost - us[i]
            bfs_copy.pop(index)
            break
            
    return us, vs      

In [ ]:
def get_ws(bfs, costs, us, vs):
    ws = []
    for i, row in enumerate(costs):
        for j, cost in enumerate(row):
            non_basic = all([p[0] != i or p[1] != j for p, v in bfs])
            if non_basic:
                ws.append(((i, j), us[i] + vs[j] - cost))
    
    return ws

In [ ]:
def can_be_improved(ws):
    for p, v in ws:
        if v > 0: return True
    return False

In [ ]:
def get_entering_variable_position(ws):
    ws_copy = ws.copy()
    ws_copy.sort(key=lambda w: w[1])
    return ws_copy[-1][0]

In [ ]:
def get_possible_next_nodes(loop, not_visited):
    last_node = loop[-1]
    nodes_in_row = [n for n in not_visited if n[0] == last_node[0]]
    nodes_in_column = [n for n in not_visited if n[1] == last_node[1]]
    if len(loop) < 2:
        return nodes_in_row + nodes_in_column
    else:
        prev_node = loop[-2]
        row_move = prev_node[0] == last_node[0]
        if row_move: return nodes_in_column
        return nodes_in_row

In [ ]:
def get_loop(bv_positions, ev_position):
    def inner(loop):
        if len(loop) > 3:
            can_be_closed = len(get_possible_next_nodes(loop, [ev_position])) == 1
            if can_be_closed: return loop
        
        not_visited = list(set(bv_positions) - set(loop))
        possible_next_nodes = get_possible_next_nodes(loop, not_visited)
        for next_node in possible_next_nodes:
            new_loop = inner(loop + [next_node])
            if new_loop: return new_loop
    
    return inner([ev_position])

In [ ]:
def loop_pivoting(bfs, loop):
    even_cells = loop[0::2]
    odd_cells = loop[1::2]
    get_bv = lambda pos: next(v for p, v in bfs if p == pos)
    leaving_position = sorted(odd_cells, key=get_bv)[0]
    leaving_value = get_bv(leaving_position)
    
    new_bfs = []
    for p, v in [bv for bv in bfs if bv[0] != leaving_position] + [(loop[0], 0)]:
        if p in even_cells:
            v += leaving_value
        elif p in odd_cells:
            v -= leaving_value
        new_bfs.append((p, v))
        
    return new_bfs

In [ ]:
def potential_method(supply, demand, costs, penalties = None):
    balanced_supply, balanced_demand, balanced_costs = get_balanced_tp(
        supply, demand, costs
    )
    def inner(bfs):
        us, vs = get_us_and_vs(bfs, balanced_costs)
        ws = get_ws(bfs, balanced_costs, us, vs)
        if can_be_improved(ws):
            ev_position = get_entering_variable_position(ws)
            loop = get_loop([p for p, v in bfs], ev_position)
            return inner(loop_pivoting(bfs, loop))
        return bfs
    
    basic_variables = inner(north_west_corner(balanced_supply, balanced_demand))
    solution = np.zeros((len(costs), len(costs[0])))
    for (i, j), v in basic_variables:
        solution[i][j] = v

    return solution

In [ ]:
def get_total_cost(costs, solution):
    total_cost = 0
    for i, row in enumerate(costs):
        for j, cost in enumerate(row):
            total_cost += cost * solution[i][j]
    return total_cost

### Проверка работы на примере из кнгиги "Выпуклый анализ" К.Ю. Осипенко

In [ ]:
costs = [
    [ 2, 1, 5, 11],
    [ 4, 3, 4, 2],
    [ 6, 2, 7, 8]
]
supply = [10, 80, 20]
demand = [40, 15, 42, 13]
solution = potential_method(supply, demand, costs)
print(solution)
print('total cost: ', get_total_cost(costs, solution))

[[10.  0.  0.  0.]
 [25.  0. 42. 13.]
 [ 5. 15.  0.  0.]]
total cost:  374.0


### Собственные данные и подсчёт времени

In [ ]:
costs = np.array([
[ 16, 19, 15, 21, 24, 44, 34, 56, 19],
[ 15, 17, 14, 19, 21, 31, 32, 62, 22],
[ 15, 17, 13, 19, 20, 26, 42, 61, 22],
[ 22, 24, 21, 20, 15, 21, 53, 44, 23],
[ 15, 14, 18, 8, 11, 32, 54, 37, 16],
[ 6, 6, 8, 13, 12, 37, 46, 43, 6],
[ 26, 22, 30, 20, 25, 47, 79, 22, 23],
[ 15, 13, 30, 11, 19, 58, 67, 32, 16],
[ 15, 10, 18, 17, 22, 46, 56, 49, 10]
])
supply = [13, 7, 14, 10, 15, 13, 9, 15, 12]
demand = [3, 9, 3, 6, 3, 12, 36, 24, 12]
solution = potential_method(supply, demand, costs)
print(solution)
print('total cost: ', get_total_cost(costs, solution)*4.7382)

[[ 0.  0.  0.  0.  0.  0. 13.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  7.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. 14.  0.  0.]
 [ 0.  0.  0.  0.  0. 10.  0.  0.  0.]
 [ 0.  2.  0.  6.  3.  2.  2.  0.  0.]
 [ 3.  0.  3.  0.  0.  0.  0.  0.  7.]
 [ 0.  0.  0.  0.  0.  0.  0.  9.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. 15.  0.]
 [ 0.  7.  0.  0.  0.  0.  0.  0.  5.]]
total cost:  12447.2514


# Transportation Problem (Simplex method)

#### Для реализования симплекс метода нам нужна каноническая формулировка транспортной задачи как ЗЛП.

In [ ]:
import numpy as np
from scipy.optimize import linprog
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Функция возвращает матрицу системы ограничений
def prepare(a, b):
    m = len(a)
    n = len(b)
    h = np.diag(np.ones(n))
    v = np.zeros((m, n))
    v[0] = 1
    for i in range(1, m):
        h = np.hstack((h, np.diag(np.ones(n))))
        k = np.zeros((m, n))
        k[i]=1
        v = np.hstack((v, k))
    return np.vstack((h, v)).astype(int), np.hstack((b,a))

In [ ]:
costs = np.array ([
    [ 2, 1, 5, 11],
    [ 4, 3, 4, 2],
    [ 6, 2, 7, 8]
])
supply = [10, 80, 20]
demand = [40, 15, 42, 13]
A = prepare(supply, demand)[0]
b = prepare(supply, demand)[1]
print('A: \n', A)
print('b: \n', b)

A: 
 [[1 0 0 0 1 0 0 0 1 0 0 0]
 [0 1 0 0 0 1 0 0 0 1 0 0]
 [0 0 1 0 0 0 1 0 0 0 1 0]
 [0 0 0 1 0 0 0 1 0 0 0 1]
 [1 1 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 1 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 1 1]]
b: 
 [40 15 42 13 10 80 20]


In [ ]:
# А также нам нужно переделать матрицу затрат в строку
c = costs.reshape(1, -1)
print('c = ', c)

c =  [[ 2  1  5 11  4  3  4  2  6  2  7  8]]


In [ ]:
def smplx(supply, demand, costs):
    a = np.copy(supply)
    b = np.copy(demand)
    c = np.copy(costs)
    # Проверяем условие замкнутости:
    if a.sum() > b.sum():
        b = np.hstack((b, [a.sum() - b.sum()]))
        c = np.hstack((c, np.zeros(len(a)).reshape(-1, 1)))
    elif a.sum() < b.sum():
        a = np.hstack((a, [b.sum() - a.sum()]))
        c = np.vstack((c, np.zeros(len(b))))
        
    m = len(a)
    n = len(b)
    A_eq, b_eq = prepare(a, b)
    
    res = linprog(c.reshape(1, -1), A_eq=A_eq, b_eq=b_eq, bounds=(0, None), method='simplex')
    return res.x.astype(int).reshape(m, n), res.fun.astype(int) # возращаем матрицу x и целевую функцию

### Проверка работы на примере из кнгиги "Выпуклый анализ" К.Ю. Осипенко

In [ ]:
costs = np.array ([
    [ 2, 1, 5, 11],
    [ 4, 3, 4, 2],
    [ 6, 2, 7, 8]
])
supply = [10, 80, 20]
demand = [40, 15, 42, 13]

x_opt, total_cost = smplx(supply, demand, costs)
print('Optimal transpotation plan: \n', x_opt)
print('Total cost: ', total_cost)
print()

Optimal transpotation plan: 
 [[10  0  0  0]
 [25  0 42 13]
 [ 5 15  0  0]]
Total cost:  374



### Собственные данные и подсчёт времени

В качестве данных для транспортной задачи выбрана следующая ситуация.
На 22 мая есть расписания поездов и самолётов, прибывающих в Москву в промежуток времени 9:50 - 10:15. Будем рассматривать автопарки "Такси Ритм" в Москве. Количество подаваемх машин расчитать сложно, но будем считать, что компания готова выделить примерно 10% от всех машин (объяснение далее). По статистике собранной Яндексом примерно 8% заказов уходят этой компании. Более того 10% от всех поездок на такси происходят из аэропортов и вокзалов (ответ на вопрос, почему компания выделяет 10%). Зная среднее количество поездок каждые пол-часа, приходящихся на этого перевозчика, можем примерно расчитать количество заказов из каждого (самого используемого по статистике) вокзала/аэропорта. Наша задача отправить в эти вокзалы/аэропорты определённое количество машин, при этом уменьшив общую стоимость затрат на поездку одного водителя. Рассчитываем расстояние до пунктов. Смотрим стоимость бензина и средний расход легквого автомобиля. Получаем соотношение киллометров в рубли. Понятно, что данные будут получены с некоторой погрешностью, но в каком-то смысле такой план подачи машин будет, быть может не самым, но всё же оптимальным.

In [ ]:
from statistics import mean
costs = np.array([
[ 16, 19, 15, 21, 24, 44, 34, 56, 19],
[ 15, 17, 14, 19, 21, 31, 32, 62, 22],
[ 15, 17, 13, 19, 20, 26, 42, 61, 22],
[ 22, 24, 21, 20, 15, 21, 53, 44, 23],
[ 15, 14, 18, 8, 11, 32, 54, 37, 16],
[ 6, 6, 8, 13, 12, 37, 46, 43, 6],
[ 26, 22, 30, 20, 25, 47, 79, 22, 23],
[ 15, 13, 30, 11, 19, 58, 67, 32, 16],
[ 15, 10, 18, 17, 22, 46, 56, 49, 10]
])
supply = [13, 7, 14, 10, 15, 13, 9, 15, 12]
demand = [3, 9, 3, 6, 3, 12, 36, 24, 12]
solution = potential_method(supply, demand, costs)
print(solution)
print('total cost: ', get_total_cost(costs, solution)*4.7382)


[[ 0.  0.  0.  0.  0.  0. 13.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  7.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. 14.  0.  0.]
 [ 0.  0.  0.  0.  0. 10.  0.  0.  0.]
 [ 0.  2.  0.  6.  3.  2.  2.  0.  0.]
 [ 3.  0.  3.  0.  0.  0.  0.  0.  7.]
 [ 0.  0.  0.  0.  0.  0.  0.  9.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. 15.  0.]
 [ 0.  7.  0.  0.  0.  0.  0.  0.  5.]]
total cost:  12447.2514


In [ ]:
times = []
for i in range (0, 20):
  start = time.time() * 1000
  smplx(supply, demand, costs)
  times.append(time.time() * 1000 - start)
times_avg = mean(times)
print ('Average simplex method time: ', times_avg, 'ms')

Average simplex method time:  50.27154541015625 ms


In [ ]:
times = []
for i in range (0, 20):
  start = time.time() * 1000
  potential_method(supply, demand, costs)
  times.append(time.time() * 1000 - start)
times_avg = mean(times)
print ('Average potential method time: ', times_avg, 'ms')

Average potential method time:  6.6680419921875 ms


We can notice the superiority of the potential method over the simplex method by almost 10 times. 